In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 5_6311828550303351502.rtf.gdoc
 a32b6b41-69b9-4299-8e3b-ecec34e68b59.pdf
 AdmitCard_80053079.pdf
'CBSE - JOINT ENTRANCE EXAMINATION (MAIN) - 2018.pdf'
'C certi.pdf'
 Classroom
 Codechef
'Colab Notebooks'
 Contacts.vcf
 data_for_colab_ritik.zip
 Datasets
'Document 4_1.jpg'
 Fake-New-LSTM-master
'fee recipient sem 4.htm'
'How to get started with Drive.pdf'
'https:  www.pointers.gdoc'
 icecream.gdoc
 IMG_20160302_094057.jpg
 IMG_20160302_094104.jpg
 IMG_20160302_094110.jpg
 IMG_20160302_094114.jpg
 IMG_20180529_084931.jpg
 IMG_20180529_113713.jpg
'JoSAA 2018: Locked Choices of the Roll Number- 28716896.pdf'
'JoSAA 7.pdf'
'JoSAA allotment .pdf'
'My Drive'
'New Doc 2019-01-12 22.22.09_1.pdf'
'New Doc 2019-01-12 22.22.51_1.pdf'
'Number Theory .gdoc'
 OD104598888636702000_invoice.pdf
 paymenthistoryredirecturl.pdf
 Resume
'Resume (1).gdoc'
'Resume (2).gdoc'
'Resume (3).gdoc'
'Resume (4).gdoc'
 Resume.gdoc
 Ritik
'Ritik adhar card.pdf'
'Ritik signature.pdf'
'riya ad

In [ ]:
data=pd.read_csv("/content/gdrive/MyDrive/Classroom/googleplaystore_user_reviews.csv")

In [ ]:
data.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [ ]:
df=pd.concat([data.Translated_Review,data.Sentiment],axis=1)
df.dropna(axis=0,inplace=True)
df.head(10)

,Translated_Review,Sentiment
0,I like eat delicious food. That's I'm cooking ...,Positive
1,This help eating healthy exercise regular basis,Positive
3,Works great especially going grocery store,Positive
4,Best idea us,Positive
5,Best way,Positive
6,Amazing,Positive
8,"Looking forward app,",Neutral
9,It helpful site ! It help foods get !,Neutral
10,good you.,Positive
11,Useful information The amount spelling errors ...,Positive


In [ ]:
df.Sentiment=[0 if i=="Positive" else 1 if i== "Negative" else 2 for i in df.Sentiment]

In [ ]:
## Get the Independent Features

X=df.drop('Sentiment',axis=1)

In [ ]:
## Get the Dependent features
y=df['Sentiment']
y

0        0
1        0
3        0
4        0
5        0
        ..
64222    0
64223    0
64226    1
64227    0
64230    1
Name: Sentiment, Length: 37427, dtype: int64

In [ ]:
X.shape

(37427, 1)

In [ ]:
y.shape

(37427,)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.5.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
### Vocabulary size
voc_size=5000

In [ ]:
messages=X.copy()

In [ ]:
messages['Translated_Review'][1]

'This help eating healthy exercise regular basis'

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    
    review = re.sub('[^a-zA-Z]', ' ', messages['Translated_Review'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

['like eat delici food cook food case best food help lot also best shelf life',
 'help eat healthi exercis regular basi',
 'work great especi go groceri store',
 'best idea us',
 'best way',
 'amaz',
 'look forward app',
 'help site help food get',
 'good',
 'use inform amount spell error question valid inform share fix star given',
 'thank great app add arthriti eye immun kidney liver detox food pleas',
 'greatest ever complet awesom maintain health must ppl love',
 'good health good health first prioriti',
 'health import world either life think',
 'mr sunita bhati thank develop make kind app realli good healthi food bodi',
 'use diabet age need control sugar thank',
 'one greatest app',
 'good nice',
 'healthi realli help',
 'god health',
 'health alway top prioriti mysg',
 'excel use',
 'found lot wealth form health',
 'found import',
 'healthi eat',
 'good simpli good',
 'test',
 'good',
 'thank advic download adob reader still get recip book',
 'recip book unabl recip book',
 'ab

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3644,
  631,
  2946,
  955,
  1516,
  955,
  1428,
  4976,
  955,
  2658,
  1275,
  826,
  4976,
  1727,
  2027],
 [2658, 631, 4759, 1564, 1781, 3295],
 [204, 4103, 4117, 516, 1854, 2712],
 [4976, 403, 2162],
 [4976, 4340],
 [2361],
 [366, 690, 392],
 [2658, 2887, 2658, 955, 2747],
 [3724],
 [2493, 941, 2235, 1158, 4800, 4668, 4935, 941, 4130, 891, 4266, 3110],
 [2357, 4103, 392, 2295, 3171, 4126, 1514, 1920, 4169, 1867, 955, 3018],
 [631, 205, 3691, 1666, 3479, 2892, 529, 1096, 4750],
 [3724, 2892, 3724, 2892, 1577, 4682],
 [2892, 1384, 3497, 4071, 2027, 4450],
 [2057, 4862, 649, 2357, 910, 1676, 889, 392, 2977, 3724, 4759, 955, 4232],
 [2493, 1202, 1019, 3098, 3161, 4865, 2357],
 [2159, 631, 392],
 [3724, 1525],
 [4759, 2977, 2658],
 [1182, 2892],
 [2892, 4041, 3263, 4682, 3429],
 [902, 2493],
 [3120, 1275, 2251, 358, 2892],
 [3120, 1384],
 [4759, 631],
 [3724, 2615, 3724],
 [1014],
 [3724],
 [2357, 733, 1555, 4710, 1863, 600, 2747, 2814, 1398],
 [2814, 1398, 4498, 2814, 1398],
 [1

### Embedding Representation

In [ ]:
sent_length=40
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4976 1727 2027]
 [   0    0    0 ... 1564 1781 3295]
 [   0    0    0 ...  516 1854 2712]
 ...
 [   0    0    0 ... 3322  886 1075]
 [   0    0    0 ...  222 4571  385]
 [   0    0    0 ... 2844 1380 4647]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0, 3644,  631, 2946,  955, 1516,  955, 1428, 4976,
        955, 2658, 1275,  826, 4976, 1727, 2027], dtype=int32)

In [ ]:
## Creating model
embedding_vector_features=200
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(3,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 40, 200)           1000000   
_________________________________________________________________
lstm_22 (LSTM)               (None, 40, 100)           120400    
_________________________________________________________________
lstm_23 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 303       
Total params: 1,201,103
Trainable params: 1,201,103
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(37427, (37427,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((37427, 40), (37427,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=32)

Epoch 1/100
784/784 [==============================] - 15s 19ms/step - loss: 0.0143 - accuracy: 0.9948 - val_loss: 0.9403 - val_accuracy: 0.8689
Epoch 2/100
784/784 [==============================] - 14s 18ms/step - loss: 0.0112 - accuracy: 0.9954 - val_loss: 1.0291 - val_accuracy: 0.8637
Epoch 3/100
784/784 [==============================] - 14s 18ms/step - loss: 0.0142 - accuracy: 0.9950 - val_loss: 0.9710 - val_accuracy: 0.8684
Epoch 4/100
784/784 [==============================] - 14s 18ms/step - loss: 0.0183 - accuracy: 0.9932 - val_loss: 0.8766 - val_accuracy: 0.8626
Epoch 5/100
784/784 [==============================] - 14s 18ms/step - loss: 0.0148 - accuracy: 0.9945 - val_loss: 0.9427 - val_accuracy: 0.8653
Epoch 6/100
784/784 [==============================] - 14s 18ms/step - loss: 0.0124 - accuracy: 0.9954 - val_loss: 0.9779 - val_accuracy: 0.8632
Epoch 7/100
784/784 [==============================] - 14s 18ms/step - loss: 0.0109 - accuracy: 0.9957 - val_loss: 1.0054 - val_ac

### Performance Metrics And Accuracy

In [ ]:
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[7244,  421,  192],
       [ 452, 2131,  151],
       [ 306,  131, 1323]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8661646830216176